In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# install dependencies: 
!pip install pyyaml==5.1
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version
import torch
assert torch.__version__.startswith("1.7")
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu102/torch1.7/index.html

     |████████████████████████████████| 274 kB 224 kB/s eta 0:00:01
  Created wheel for pyyaml: filename=PyYAML-5.1-cp37-cp37m-linux_x86_64.whl size=44074 sha256=66799ee69cd329844bd4b8b00e5570e6c49a9af40604983f896e899745784355
  Stored in directory: /root/.cache/pip/wheels/77/f5/10/d00a2bd30928b972790053b5de0c703ca87324f3fead0f2fd9
Successfully built pyyaml
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 5.3.1
ERROR: Cannot uninstall 'PyYAML'. It is a distutils installed project and thus we cannot accurately determine which files belong to it which would lead to only a partial uninstall.
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.
1.7.0 True
gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

Looking in links: https://

  Created wheel for pycocotools: filename=pycocotools-2.0.2-cp37-cp37m-linux_x86_64.whl size=272627 sha256=acb4ff1bb225567622d6893a2fff9a99fa8adff16e2c3e57480b09eb2fbbd239
  Stored in directory: /root/.cache/pip/wheels/bc/cf/1b/e95c99c5f9d1648be3f500ca55e7ce55f24818b0f48336adaf
Successfully built fvcore iopath pycocotools
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [3]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()


# import some common libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import  StratifiedShuffleSplit
import os, json, cv2, random

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog 

import copy
import logging
import numpy as np
from typing import Callable, List, Union
import torch

from detectron2.config import configurable
from detectron2.data import MetadataCatalog
from detectron2.data import detection_utils as utils
from detectron2.data import transforms as T

from detectron2.data import detection_utils as utils
import copy
import detectron2.data.transforms as T
import matplotlib.pyplot as plt
from detectron2.data import DatasetMapper
import torch
import os
import numpy as np

from detectron2.config import configurable

In [18]:
import pickle

In [4]:
dirpath = '../input/vinbigdata-competition-jpg-data-3x-downsampled'

df = pd.read_csv(f'{dirpath}/train_downsampled.csv')
df.shape

(67914, 8)

In [5]:
df.head()

,image_id,class_name,class_id,rad_id,x_min,y_min,x_max,y_max
0,50a418190bc3fb1ef1633bf9678929b3,No finding,14,R11,NaN,NaN,NaN,NaN
1,21a10246a5ec7af151081d0cd6d65dc9,No finding,14,R7,NaN,NaN,NaN,NaN
2,9a5094b2563a1ef3ff50dc5c7ff71345,Cardiomegaly,3,R10,230.0,458.0,551.0,610.0
3,051132a778e61a86eb147c7c6f564dfe,Aortic enlargement,0,R10,421.0,247.0,537.0,339.0
4,063319de25ce7edb9b1c6b8881290140,No finding,14,R10,NaN,NaN,NaN,NaN


In [6]:
df['w'], df['h'] = df['x_max'] - df['x_min'], df['y_max'] - df['y_min']
df['area'] = df['w'] * df['h']
df.head()

,image_id,class_name,class_id,rad_id,x_min,y_min,x_max,y_max,w,h,area
0,50a418190bc3fb1ef1633bf9678929b3,No finding,14,R11,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,21a10246a5ec7af151081d0cd6d65dc9,No finding,14,R7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,9a5094b2563a1ef3ff50dc5c7ff71345,Cardiomegaly,3,R10,230.0,458.0,551.0,610.0,321.0,152.0,48792.0
3,051132a778e61a86eb147c7c6f564dfe,Aortic enlargement,0,R10,421.0,247.0,537.0,339.0,116.0,92.0,10672.0
4,063319de25ce7edb9b1c6b8881290140,No finding,14,R10,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=0)
print(df.shape)
df_dd = df.drop_duplicates('image_id')
print(df_dd.shape)
df_dd = df_dd.reset_index()
sss.get_n_splits(df_dd['image_id'], df_dd['class_id'])
for train_index, test_index in sss.split(df_dd['image_id'], df_dd['class_id']):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = df_dd['image_id'][train_index], df_dd['image_id'][test_index]
    y_train, y_test = df_dd['class_id'][train_index], df_dd['class_id'][test_index]

(67914, 11)
(15000, 11)
TRAIN: [10223 11258 11436 ... 12186  5843 12594] TEST: [12600  6685 12389 ... 13066  3822   223]


In [8]:
classes = df.drop_duplicates('class_id').sort_values('class_id')[['class_name']].values[:-1].ravel().tolist()
print(classes)
thing_classes = {class_name: index for index, class_name in enumerate(classes)}
print(thing_classes)

['Aortic enlargement', 'Atelectasis', 'Calcification', 'Cardiomegaly', 'Consolidation', 'ILD', 'Infiltration', 'Lung Opacity', 'Nodule/Mass', 'Other lesion', 'Pleural effusion', 'Pleural thickening', 'Pneumothorax', 'Pulmonary fibrosis']
{'Aortic enlargement': 0, 'Atelectasis': 1, 'Calcification': 2, 'Cardiomegaly': 3, 'Consolidation': 4, 'ILD': 5, 'Infiltration': 6, 'Lung Opacity': 7, 'Nodule/Mass': 8, 'Other lesion': 9, 'Pleural effusion': 10, 'Pleural thickening': 11, 'Pneumothorax': 12, 'Pulmonary fibrosis': 13}


## Read in dicom files

In [35]:
X_test

12600    f87d3fef16e2bb2ef72179864c9f6cef
6685     1a15da91ec46d4887b40c65866fba831
12389    95badc82bced45380cf4275bb11d2bb9
7013     2a4a459820226a8b7683df0df4adac6f
2310     73b83ade1fb63fc7c966c1d776323ad0
                       ...               
2636     bde4bfa149bc13cee2de7c2e942979e7
1808     fe358fa490f9b97a3be8142961dd69f6
13066    97fd11ea81a44af39e0f8bac372acb75
3822     0fdba5b4298773f488709251e49b26a2
223      6a956ce4e56235dd8de081a965a36c2a
Name: image_id, Length: 3000, dtype: object

In [30]:
##### DATASET PREPARING: CONERTING CSV TO DICTIONARY WHICH CONTAINS ANNOTATION AND IMAGE PATH ETC.

from detectron2.structures import BoxMode
from pathlib import Path

def chest_dicts_train(images, img_dir = '../input/vinbigdata-competition-jpg-data-3x-downsampled/train/train', load_cache=True):
    cache_path = Path(".") / f"dataset_dicts_cache_train.pkl"
    if load_cache == False:
        dataset_dicts = []
        for idx, v in enumerate(images):
            record = {}

            filename = os.path.join(img_dir, v + '.jpg')

            image = cv2.imread(filename)
            height, width, ch = image.shape

            record["file_name"] = filename
            record["image_id"] = idx
            record["height"] = height # RANDOM Not Req
            record["width"] = width # RANDOM Not Req

            annos = df[df.image_id == v]
            objs = []
            for _, anno in annos.iterrows():
                if anno.class_id != 14:

                    obj = {
                        "bbox": [int(anno.x_min), int(anno.y_min), int(anno.w), int(anno.h)],
                        "bbox_mode": BoxMode.XYWH_ABS,
                        "category_id": int(anno.class_id)
                    }
                    objs.append(obj)
            record["annotations"] = objs
            dataset_dicts.append(record)
            
        with open(cache_path, mode="wb") as f:
                pickle.dump(dataset_dicts, f)
    else:
        with open(cache_path, mode="rb") as f:
            dataset_dicts = pickle.load(f)
    return dataset_dicts

def chest_dicts_test(images, img_dir = '../input/vinbigdata-competition-jpg-data-3x-downsampled/train/train', load_cache=True):
    cache_path = Path(".") / f"dataset_dicts_cache_test.pkl"
    if load_cache == False:
        dataset_dicts = []
        for idx, v in enumerate(images):
            record = {}

            filename = os.path.join(img_dir, v + '.jpg')

            image = cv2.imread(filename)
            height, width, ch = image.shape

            record["file_name"] = filename
            record["image_id"] = idx + 12000
            record["height"] = height # RANDOM Not Req
            record["width"] = width # RANDOM Not Req

            annos = df[df.image_id == v]
            objs = []
            for _, anno in annos.iterrows():
                if anno.class_id != 14:

                    obj = {
                        "bbox": [int(anno.x_min), int(anno.y_min), int(anno.w), int(anno.h)],
                        "bbox_mode": BoxMode.XYWH_ABS,
                        "category_id": int(anno.class_id)
                    }
                    objs.append(obj)
            record["annotations"] = objs
            dataset_dicts.append(record)
        with open(cache_path, mode="wb") as f:
                pickle.dump(dataset_dicts, f)
    else:
        with open(cache_path, mode="rb") as f:
            dataset_dicts = pickle.load(f)
    return dataset_dicts
def train():
    return chest_dicts_train(X_train, load_cache=True)
def val():
    return chest_dicts_test(X_test, load_cache=True)


In [11]:
DatasetCatalog.register("chest_Train", train)
MetadataCatalog.get("chest_Train").thing_classes=classes
DatasetCatalog.register("chest_Val",val)
MetadataCatalog.get("chest_Val").thing_classes=classes

In [12]:
"""
To calculate & record validation loss

Original code from https://medium.com/@apofeniaco/training-on-detectron2-with-a-validation-set-and-plot-loss-on-it-to-avoid-overfitting-6449418fbf4e
by @apofeniaco
"""
import numpy as np
import logging

from detectron2.engine.hooks import HookBase
from detectron2.utils.logger import log_every_n_seconds
import detectron2.utils.comm as comm
import torch
import time
import datetime


class LossEvalHook(HookBase):
    def __init__(self, eval_period, model, data_loader):
        self._model = model
        self._period = eval_period
        self._data_loader = data_loader

    def _do_loss_eval(self):
        # Copying inference_on_dataset from evaluator.py
        total = len(self._data_loader)
        num_warmup = min(5, total - 1)

        start_time = time.perf_counter()
        total_compute_time = 0
        losses = []
        for idx, inputs in enumerate(self._data_loader):
            if idx == num_warmup:
                start_time = time.perf_counter()
                total_compute_time = 0
            start_compute_time = time.perf_counter()
            if torch.cuda.is_available():
                torch.cuda.synchronize()
            total_compute_time += time.perf_counter() - start_compute_time
            iters_after_start = idx + 1 - num_warmup * int(idx >= num_warmup)
            seconds_per_img = total_compute_time / iters_after_start
            if idx >= num_warmup * 2 or seconds_per_img > 5:
                total_seconds_per_img = (time.perf_counter() - start_time) / iters_after_start
                eta = datetime.timedelta(seconds=int(total_seconds_per_img * (total - idx - 1)))
                log_every_n_seconds(
                    logging.INFO,
                    "Loss on Validation  done {}/{}. {:.4f} s / img. ETA={}".format(
                        idx + 1, total, seconds_per_img, str(eta)
                    ),
                    n=5,
                )
            loss_batch = self._get_loss(inputs)
            losses.append(loss_batch)
        mean_loss = np.mean(losses)
        # self.trainer.storage.put_scalar('validation_loss', mean_loss)
        comm.synchronize()

        # return losses
        return mean_loss

    def _get_loss(self, data):
        # How loss is calculated on train_loop
        metrics_dict = self._model(data)
        metrics_dict = {
            k: v.detach().cpu().item() if isinstance(v, torch.Tensor) else float(v)
            for k, v in metrics_dict.items()
        }
        total_losses_reduced = sum(loss for loss in metrics_dict.values())
        return total_losses_reduced

    def after_step(self):
        next_iter = int(self.trainer.iter) + 1
        is_final = next_iter == self.trainer.max_iter
        if is_final or (self._period > 0 and next_iter % self._period == 0):
            mean_loss = self._do_loss_eval()
            self.trainer.storage.put_scalars(validation_loss=mean_loss)
            print("validation do loss eval", mean_loss)
        else:
            pass
            # self.trainer.storage.put_scalars(timetest=11)

In [13]:
from detectron2.data import detection_utils as utils
from detectron2.data import build_detection_train_loader
from PIL import Image
# writing a new DatasetMapper as we have dicom files

class DatasetMapper:
    
    

    @configurable
    def __init__(
        self,
        is_train: bool,
        *,
        augmentations: List[Union[T.Augmentation, T.Transform]],
        image_format: str = 'BGR',
        use_instance_mask: bool = False,
        use_keypoint: bool = False,
        instance_mask_format = "polygon",
        keypoint_hflip_indices = None,
        precomputed_proposal_topk= None,
        recompute_boxes: bool = False,
    ):
        """
        NOTE: this interface is experimental.

        Args:
            is_train: whether it's used in training or inference
            augmentations: a list of augmentations or deterministic transforms to apply
            image_format: an image format supported by :func:`detection_utils.read_image`.
            use_instance_mask: whether to process instance segmentation annotations, if available
            use_keypoint: whether to process keypoint annotations if available
            instance_mask_format: one of "polygon" or "bitmask". Process instance segmentation
                masks into this format.
            keypoint_hflip_indices: see :func:`detection_utils.create_keypoint_hflip_indices`
            precomputed_proposal_topk: if given, will load pre-computed
                proposals from dataset_dict and keep the top k proposals for each image.
            recompute_boxes: whether to overwrite bounding box annotations
                by computing tight bounding boxes from instance mask annotations.
        """

        # fmt: off
        self.is_train               = is_train
        self.augmentations          = T.AugmentationList(  augmentations)
        self.image_format           = image_format
        
        # fmt: on
        logger = logging.getLogger(__name__)
        mode = "training" if is_train else "inference"
        logger.info(f"[DatasetMapper] Augmentations used in {mode}: {augmentations}")

    @classmethod
    def from_config(cls, cfg, is_train: bool = True):
        augs = utils.build_augmentation(cfg, is_train)
        if cfg.INPUT.CROP.ENABLED and is_train:
            augs.insert(0, T.RandomCrop(cfg.INPUT.CROP.TYPE, cfg.INPUT.CROP.SIZE))
            recompute_boxes = cfg.MODEL.MASK_ON
        else:
            recompute_boxes = False


        ret = {
            "is_train": is_train,
            "augmentations": augs,
            "image_format": cfg.INPUT.FORMAT,
            "use_instance_mask": cfg.MODEL.MASK_ON,
            "instance_mask_format": cfg.INPUT.MASK_FORMAT,
            "use_keypoint": cfg.MODEL.KEYPOINT_ON,
            "recompute_boxes": recompute_boxes,
        }

        return ret

    def __call__(self, dataset_dict):
        # print(dataset_dict)
        dataset_dict = copy.deepcopy(dataset_dict)
        image = utils.read_image(dataset_dict["file_name"])
        
        # I'm not sure about these augmentations, need to properly go over them
        auginput = T.AugInput(image)
        transform = self.augmentations(auginput)
        image = np.expand_dims(auginput.image, axis=2).copy()
        image = torch.from_numpy(image.transpose(2, 0, 1))
        annos = [
            utils.transform_instance_annotations(annotation, [transform], image.shape[1:])
            for annotation in dataset_dict.pop("annotations")
        ]
        return {
        # create the format that the model expects
        "image": image,
        "instances": utils.annotations_to_instances(annos, image.shape[1:])
        }

In [44]:
from detectron2.engine import DefaultTrainer
from detectron2.data import MetadataCatalog, build_detection_train_loader,build_detection_test_loader
from detectron2.evaluation import COCOEvaluator
from detectron2.engine import DefaultPredictor
from detectron2.evaluation import COCOEvaluator, inference_on_dataset

In [87]:
class Trainer(DefaultTrainer):
    
    
#     @classmethod
#     def build_evaluator(cls, cfg, dataset_name, output_folder=None):
#         if output_folder is None:
#             output_folder = os.path.join(cfg.OUTPUT_DIR, "inference")
#         # return COCOEvaluator(dataset_name, cfg, True, output_folder)
#         return COCOEvaluator(dataset_name, tasks=("bbox"), distributed=False, output_dir=output_folder)
    
#     @classmethod
#     def build_test_loader(cls, cfg, dataset_name):
#         return build_detection_test_loader(cfg,dataset_name,
#                 mapper=DatasetMapper(cfg, is_train = False))
    
    @classmethod
    def build_test_loader(cls, cfg, dataset_name):
        return build_detection_test_loader(cfg,dataset_name,
                mapper=DatasetMapper(cfg, is_train = False))
    

    @classmethod
    def build_train_loader(cls, cfg):
        return build_detection_train_loader(dataset =train(),
                mapper=DatasetMapper(cfg, is_train = True),
                aspect_ratio_grouping=False, 
                total_batch_size = cfg.SOLVER.IMS_PER_BATCH)
    
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        return COCOEvaluator(dataset_name, ("bbox",), False, output_dir=output_folder)

    
    def build_hooks(self):
        hooks = super(Trainer, self).build_hooks()
        cfg = self.cfg
        if len(cfg.DATASETS.TEST) > 0:
            loss_eval_hook = LossEvalHook(
                cfg.TEST.EVAL_PERIOD,
                self.model,
                Trainer.build_test_loader(cfg, cfg.DATASETS.TEST[0]),
            )
            hooks.insert(-1, loss_eval_hook)

        return hooks
    
    
    

In [88]:
Batch = 10
Epochs = 1
steps = 120  #  ### INCREASE THE STEPS   (len(X_train) // Batch) * Epochs 
cfg = get_cfg()
NAME = "COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"
cfg.merge_from_file(model_zoo.get_config_file(NAME))
cfg.DATASETS.TRAIN = ("chest_Train",)
cfg.DATASETS.TEST = ("chest_Val", )
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS =  model_zoo.get_checkpoint_url(NAME)
cfg.SOLVER.IMS_PER_BATCH = Batch
cfg.CUDNN_BENCHMARK =  True
cfg.MODEL.RETINANET.NUM_CLASSES  = len(classes)
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.LR_SCHEDULER_NAME = "WarmupCosineLR"
cfg.SOLVER.MAX_ITER = steps  
cfg.OUTPUT_DIR = './output'
cfg.MODEL.PIXEL_MEAN = [103.530]
cfg.MODEL.PIXEL_STD = [1.0]
cfg.SOLVER.CHECKPOINT_PERIOD = 1000
cfg.SOLVER.CLIP_GRADIENTS.CLIP_VALUE = 0.95
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

In [89]:
trainer = Trainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()
#     cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
#     print("Original thresh", cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST)  # 0.05
#     cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.0  # set a custom testing threshold
#     print("Changed  thresh", cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST)
#     predictor = DefaultPredictor(cfg)
#     trainer.test(cfg, trainer.model)


    
    
#    evaluator = COCOEvaluator("chest_Val", ("bbox", ), False, output_dir="./output/")
#     val_loader = build_detection_test_loader(cfg, "chest_Val", mapper=DatasetMapper(cfg, is_train = False))
#     print(inference_on_dataset(trainer.model, val_loader, evaluator))
    
    

[02/07 02:49:35 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

[02/07 02:49:35 d2.data.common]: Serializing 12000 elements to byte tensors and concatenating them all ...
[02/07 02:49:35 d2.data.common]: Serialized dataset takes 3.16 MiB
[02/07 02:50:32 d2.data.common]: Serializing 3000 elements to byte tensors and concatenating them all ...
[02/07 02:50:33 d2.data.common]: Serialized dataset takes 0.79 MiB
[02/07 02:50:33 d2.engine.train_loop]: Starting training from iteration 0
[02/07 02:50:57 d2.utils.events]:  eta: 0:24:26  iter: 19  total_loss: 0.597  loss_cls: 0.08146  loss_box_reg: 0.004429  loss_rpn_cls: 0.471  loss_rpn_loc: 0.04155  time: 1.2432  data_time: 0.1215  lr: 4.9922e-06  max_mem: 12171M
[02/07 02:51:22 d2.utils.events]:  eta: 0:23:59  iter: 39  total_loss: 0.563  loss_cls: 0.1247  loss_box_reg: 0.01069  loss_rpn_cls: 0.4283  loss_rpn_loc: 0.03273  time: 1.2443  data_time: 0.1313  lr: 9.9642e-06  max_mem: 12171M
[02/07 02:51:47 d2.utils.events]:  eta: 0:23:34  iter: 59  total_loss: 0.7893  loss_cls: 0.109  loss_box_reg: 0.007446  

RuntimeError: CUDA out of memory. Tried to allocate 776.00 MiB (GPU 0; 15.90 GiB total capacity; 11.50 GiB already allocated; 435.75 MiB free; 14.58 GiB reserved in total by PyTorch)